## Analyze whether TOBS varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSBSSSB'
meas='TOBS'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
#df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
1186


In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [6]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_2 FROM weather


,station,year,coeff_2
0,USC00045915,2009.0,-189.460118
1,USC00049423,1970.0,-6.243226
2,USC00040227,1968.0,-95.950107
3,USC00045032,1939.0,-338.001572
4,USC00046502,1955.0,-366.881351


In [12]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.tail(5)

station,USC00040227,USC00040232,USC00040693,USC00041043,USC00041206,USC00041967,USC00042177,USC00042580,USC00043244,USC00043650,...,USC00047880,USC00048557,USC00048560,USC00048562,USC00048999,USC00049001,USC00049185,USC00049423,USW00023211,USW00023258
year,,,,,,,,,,,,,,,,,,,,,
2008.0,NaN,NaN,NaN,NaN,NaN,46.647121,NaN,NaN,183.696353,NaN,...,119.038092,NaN,NaN,NaN,NaN,-114.671502,NaN,NaN,NaN,NaN
2009.0,NaN,NaN,NaN,NaN,NaN,126.366504,NaN,NaN,313.206253,NaN,...,134.607359,NaN,-298.411732,NaN,NaN,-57.861450,NaN,NaN,NaN,NaN
2010.0,NaN,NaN,NaN,NaN,NaN,141.852369,NaN,NaN,351.102826,NaN,...,285.077715,NaN,NaN,NaN,NaN,-84.915886,NaN,NaN,NaN,NaN
2011.0,NaN,NaN,NaN,NaN,NaN,140.881131,NaN,NaN,370.031809,NaN,...,NaN,NaN,NaN,NaN,NaN,-53.012720,NaN,NaN,NaN,NaN
2012.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,458.637263,NaN,...,NaN,NaN,NaN,NaN,NaN,-2.233927,NaN,NaN,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [11]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

RMS_total = RMS(year_station_table)
RMS_year = RMS(tbl_minus_year)
RMS_station = RMS(tbl_minus_station)

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

print 'Reduction of RMS by mean-by-station', RMS_station/RMS_total
print 'Reduction of RMS by mean-by-year', RMS_year/RMS_total

total RMS                   =  255.753539901
RMS removing mean-by-station=  132.020692328
RMS removing mean-by-year   =  236.585402855
Reduction of RMS by mean-by-station 0.516202795781
Reduction of RMS by mean-by-year 0.925052309918


In [9]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 255.753539901
0 after removing mean by year    = 236.585402855
0 after removing mean by stations= 113.382448772
1 after removing mean by year    = 106.991931754
1 after removing mean by stations= 106.107450059
2 after removing mean by year    = 105.885773374
2 after removing mean by stations= 105.814748195
3 after removing mean by year    = 105.789674164
3 after removing mean by stations= 105.780420495
4 after removing mean by year    = 105.776912864
4 after removing mean by stations= 105.775555211
